In [1]:
import warnings
import pandas as pd
from pathlib import Path
warnings.filterwarnings('ignore')


# 远程电脑关机，营业周期 指标 暂时在这里提取
operating = pd.read_excel('./data/补充数据/经营能力.xlsx', skiprows=[0, 2], dtype={'股票代码': 'object'}, parse_dates=['统计截止日期'], engine="openpyxl")
operating


,股票代码,股票简称,统计截止日期,报表类型编码,公告来源,营业周期A,营业周期B,营业周期C,营业周期TTM
0,000001,平安银行,2013-12-31,A,0,1364.205490,1035.055780,1035.055780,1035.055780
1,000001,平安银行,2014-03-31,A,0,1146.102614,1155.965424,1155.965424,1110.887919
2,000001,平安银行,2014-06-30,A,0,1210.305652,1140.452022,1140.452022,1144.742496
3,000001,平安银行,2014-09-30,A,0,1238.387284,1128.598182,1128.598182,1179.470111
4,000001,平安银行,2014-12-31,A,0,1323.763550,1157.201917,1157.201917,1157.201917
...,...,...,...,...,...,...,...,...,...
338818,920128,胜业电气,2023-06-30,B,1,NaN,NaN,NaN,NaN
338819,920128,胜业电气,2023-12-31,A,1,209.190445,104.595217,202.149554,202.149554
338820,920128,胜业电气,2023-12-31,B,1,196.968640,98.484324,189.357525,189.357525
338821,920128,胜业电气,2024-06-30,A,1,202.222463,101.111220,200.536911,101.811449


In [6]:
operating = operating[
    (operating['公告来源'] == 0) & (operating['报表类型编码'] == 'A') & (operating['统计截止日期'].dt.month == 12)]
operating["123"]=operating["营业周期A"]+operating["营业周期B"]
operating["123"].fillna(0, inplace=True)
def add(*args,**kwargs):
    return args+1

operating["123"].apply(lambda x: x+1, inplace=True)
operating

,股票代码,股票简称,统计截止日期,报表类型编码,公告来源,营业周期A,营业周期B,营业周期C,营业周期TTM,123
0,000001,平安银行,2013-12-31,A,0,1364.205490,1035.055780,1035.055780,1035.055780,2399.261270
4,000001,平安银行,2014-12-31,A,0,1323.763550,1157.201917,1157.201917,1157.201917,2480.965467
8,000001,平安银行,2015-12-31,A,0,1237.304913,1122.976719,1122.976719,1122.976719,2360.281632
12,000001,平安银行,2016-12-31,A,0,1471.646676,1286.594111,1286.594111,1283.078827,2758.240787
16,000001,平安银行,2017-12-31,A,0,1479.955723,1470.623786,1470.623786,1470.623786,2950.579509
...,...,...,...,...,...,...,...,...,...,...
338625,900957,凌云B股,2020-12-31,A,0,764.733660,710.017847,710.017847,708.077908,1474.751507
338632,900957,凌云B股,2021-12-31,A,0,858.338820,764.405910,764.405910,764.405910,1622.744730
338639,900957,凌云B股,2022-12-31,A,0,888.529690,848.945795,848.945795,848.945795,1737.475485
338647,900957,凌云B股,2023-12-31,A,0,977.543286,922.688791,922.688791,922.688791,1900.232077


In [5]:
operating[['股票代码', '营业周期A']]

,股票代码,营业周期A
0,000001,1364.205490
4,000001,1323.763550
8,000001,1237.304913
12,000001,1471.646676
16,000001,1479.955723
...,...,...
338625,900957,764.733660
338632,900957,858.338820
338639,900957,888.529690
338647,900957,977.543286


In [ ]:
operating = operating[
    (operating['公告来源'] == 0) & (operating['报表类型编码'] == 'A') & (operating['统计截止日期'].dt.month == 12)]
operating['截止日期'] = operating['统计截止日期'].dt.year.astype('int')
operating = operating[["股票代码", "截止日期", "营业周期B"]]
dataset = pd.read_csv('./data/2014-2023目标企业绩效指标数据(未处理).csv', dtype={"股票代码": 'object'})
operating.drop_duplicates(subset=["股票代码", "截止日期"],inplace=True)
dataset = pd.merge(dataset, operating, how='left', on=['股票代码', '截止日期'])
print("共载入企业绩效数据 {} 条".format(dataset.shape[0]))
# 读取载入补充指标
dataset.drop(["纳税总额", "社会捐赠额"], axis=1, inplace=True)
operating = pd.read_csv('./data/补充数据.csv', dtype={"股票代码": 'object'})
operating.drop_duplicates(subset=["股票代码", "截止日期"],inplace=True)
dataset = pd.merge(dataset, operating, how='left', on=['股票代码', '截止日期'])
del operating
# 读取载入数字化转型水平
digital = pd.read_csv('./data/数字化单位及数字化程度(未筛选).csv', dtype={"股票代码": 'object', "行业代码": "object"},parse_dates=['上市日期'])
digital.drop_duplicates(subset=["股票代码", "截止日期"],inplace=True)
print("共载入数字化转型水平数据 {} 条".format(digital.shape[0]))
dataset = pd.merge(digital, dataset, how='left', on=['股票代码', '截止日期'])
dataset['行业代码'] = dataset['行业代码'].apply(lambda x: int(x.split('.')[0]))
print(f"")
del digital
print("累计获取企业样本数据 {0} 条，其中2017年以后样本{1}个".format(dataset.shape[0], dataset[dataset["截止日期"]>2017].shape[0]))
# 筛选制造业
dataset = dataset[(dataset['行业代码']>12)&(dataset['行业代码']<44)& (dataset['行业代码'] != 39)].drop_duplicates(subset=["股票代码", "截止日期"])

# 剔除存在st状态企业
dscids = []
# 剔除st企业：这里主要剔除，2020年以后三年内企业状态存在非正常的企业，只要存在一年就剔除
for scid in set(dataset["股票代码"].tolist()):
    cdata = dataset[(dataset["股票代码"] == scid) & (dataset["截止日期"] > 2017)]
    flag = cdata[~(cdata["上市状态"] == "正常上市") | (cdata["股票简称"].str.contains("ST"))].shape[0]
    if flag:
        dscids.append(scid)
dataset = dataset[~dataset["股票代码"].isin(dscids)]
print(f"剔除非制造业以及计算机制造业后剩余样本{dataset[dataset['截止日期']>2017].shape[0]}")
data = dataset.copy(deep=True)
# print(set(dataset["股票代码"].tolist()))

In [3]:
import os
import numpy as np
from utils.config import fill_na

dataset = data.copy(deep=True)
# 指标清洗
dataset = dataset[(dataset['截止日期'] > 2014) & (~dataset['股票简称'].isna())]
dataset['行业代码'] = dataset['行业代码'].apply(lambda x: int(float(x)))
dataset["股票简称"] = dataset["股票简称"].apply(lambda x: x.upper())

dataset.loc[:, '两职合一'].replace({2: 0}, inplace=True)
eq_zero = ['发明专利', '专利', 'wufei', 'yuanchun', 'zhaochenyv', 'lishouxi', '两职合一', '计划投资项目数', '计划投资总金额', '是否经第三方机构审验', '是否参照GRI《可持续发展报告指南》', '是否披露股东权益保护',
'是否披露债权人权益保护', '是否披露职工权益保护', '是否披露供应商权益保护', '是否披露客户及消费者权益保护',
'是否披露环境和可持续发展', '是否披露公共关系和社会公益事业', '是否披露社会责任制度建设及改善措施',
'是否披露安全生产内容', '是否披露公司存在的不足', '公告披露意愿', '审计师是否来自四大会计师事务所',
'供应链合作伙伴数量']



dataset= fill_na(dataset)
# 部分数据空值等价于0
for name in eq_zero:
    dataset.loc[:, name].fillna(0, inplace=True)
for name in ["员工数目", "董事人数", "独董人数", '发明专利', '两职合一']:
    dataset.loc[:, name].fillna(-1, inplace=True)
    dataset[name] = dataset[name].apply(lambda x: int(x) if x != -1 else pd.NA)
    
# 提取部分指标
dataset['发明专利申请数'] = dataset['发明专利'].apply(lambda x: np.log(1 + x))
dataset['独董比例(%)'] = dataset['独董人数'] / dataset['董事人数']
dataset['现金资产比率(%)'] = dataset['经营活动产生的现金流量净额'] / dataset['总资产']
dataset['员工人均营收比(%)'] = dataset['营业总收入'] / (dataset['员工数目'] * 1e6)
dataset['提供岗位增长率(%)'] = dataset.groupby('股票代码')['员工数目'].pct_change(1, fill_method='ffill')
dataset['员工收入增长率(%)'] = dataset.groupby('股票代码')['应付职工薪酬'].pct_change(1, fill_method='ffill')
dataset['董事会规模'] = dataset['董事人数'].apply(lambda x: np.log(1 + x))
dataset['董事会规模'].replace(to_replace=np.inf, value=pd.NA, inplace=True)
dataset['提供岗位增长率(%)'].replace(to_replace=np.inf, value=pd.NA, inplace=True)
dataset['员工收入增长率(%)'].replace(to_replace=np.inf, value=pd.NA, inplace=True)
dataset['员工收入增长率(%)'] = dataset['员工收入增长率(%)'] * 100
dataset['提供岗位增长率(%)'] = dataset['提供岗位增长率(%)'] * 100
dataset['固定资产数字化'] = dataset['固定资产数字化'] * 100
dataset['无形资产数字化'] = dataset['无形资产数字化'] * 100
dataset["数字化软硬件投入比(%)"] = (dataset["固定资产数字化"] + dataset["无形资产数字化"]) / 2
for name in ('纳税总额', '每股社会贡献值', '社会捐赠额', '发布需求次数', '招聘人数'):
    dataset.loc[:, name].fillna(0, inplace=True)
    
dataset["每股社会贡献值"] = dataset["每股社会贡献值"] / 27
dataset['纳税营收比'] = dataset['纳税总额'] / dataset['营业总收入']
dataset['捐献营收比'] = dataset['社会捐赠额'] / dataset['营业总收入']
# 将纳税营收比和捐献营收比最大值标准化为2
tax, devo = max(dataset['纳税营收比']), max(dataset['捐献营收比'])
dataset['纳税营收比'] = dataset['纳税营收比'] / tax * 2
dataset['捐献营收比'] = dataset['捐献营收比'] / devo * 2
csr = ['纳税营收比', '捐献营收比', '是否经第三方机构审验', '是否参照GRI《可持续发展报告指南》', '是否披露股东权益保护','是否披露债权人权益保护', '是否披露职工权益保护', '是否披露供应商权益保护', '是否披露客户及消费者权益保护','是否披露环境和可持续发展', '是否披露公共关系和社会公益事业', '是否披露社会责任制度建设及改善措施','是否披露安全生产内容', '是否披露公司存在的不足', '公告披露意愿', '审计师是否来自四大会计师事务所',"每股社会贡献值"]

dataset["社会责任报告质量"] = dataset[csr].sum(axis=1)
dataset["数字人力投入计划"] = dataset['发布需求次数'] + dataset['招聘人数'] * 2
dataset["数字创新标准"] = dataset['当年参与国家标准制定数量'] * 5 + dataset['当年参与行业标准制定数量']
dataset['计划投资项目数'] = (dataset['计划投资项目数'] - min(dataset['计划投资项目数'])) / (
            max(dataset['计划投资项目数']) - min(dataset['计划投资项目数'])) * 100
dataset['计划投资总金额'] = (dataset['计划投资总金额'] - min(dataset['计划投资总金额'])) / (
            max(dataset['计划投资总金额']) - min(dataset['计划投资总金额'])) * 100

dataset["数字资本计划投入"] = (dataset['计划投资项目数'] + dataset['计划投资总金额']) / 2

dataset.drop(['发明专利', '专利', "独董人数", '董事人数', "经营活动产生的现金流量净额", "总资产", "营业总收入", '应付职工薪酬', "上市日期", "上市状态",'员工数目', "托宾Q值", '发布需求次数', '招聘人数', '当年参与国家标准制定数量','当年参与行业标准制定数量', '纳税总额', '社会捐赠额', '固定资产数字化', '无形资产数字化', '计划投资项目数', '计划投资总金额', *csr], axis=1, inplace=True)
# 线性插补获取数据
dataset= fill_na(dataset)

In [6]:
rename_map = {
    "股权性质编码": '股权性质',
    "数字技术应用z": "数字技术应用",
    "互联网商业模式z": "商业模式变革",
    "智能制造z": "智能制造",
    "现代信息系统z": "数字化供应链覆盖度",
    "资产负债率": "资产负债率(%)",
    "速动比率": "速动比率(%)",
    '经营活动产生的现金流量净额／流动负债': "现金流动负债比率(%)",
    "长期资本负债率": "长期资本负债率(%)",
    '权益乘数': '权益乘数(%)',
    "营业总收入增长率": "营业收入增长率(%)",
    "资本保值增值率": "资本保值增值率(%)",
    "总资产增长率": "总资产增长率(%)",
    "资本积累率": "资本积累率(%)",
    "营业利润增长率": "营业利润增长率(%)",
    "净资产收益率(ROE)": "净资产收益率(%)",
    '销售期间费用率': '销售期间费用率(%)',
    "资产报酬率": "资产报酬率(%)",
    "EBITDA率": "EBITDA率(%)",
    "成本费用利润率": "成本费用利润率(%)",
    "营业利润率": "营业利润率(%)",
    "投入资本回报率(ROIC)": "投入资本回报率(%)",
    "资本收益率": "资本收益率(%)",
    '营业周期B':'营业周期',
    '总资产周转率':'总资产周转率(%)',
    '应收账款周转率':'应收账款周转率(%)',
    '流动资产周转率':'流动资产周转率(%)',
    '存货周转率':'存货周转率(%)',
    
    "研发人员数量占比(%)": "技术人员占比(%)",
    "研发投入占营业收入比例(%)": "研发营收比(%)",
    '管理层数字职务设立数': '管理层数字职务设立',
    '创新导向前瞻性':'数字化战略导向前瞻性',
    '创新导向持续性':'数字化战略导向持续性',
    '创新导向广度': '数字化战略导向广度',
    '创新导向强度':'数字化战略导向强度',
    '累计参与国家级科技创新基地建设数量':'科技创新基地建设',
    '计划投资项目数':"数字资本计划投入",
    '数字人力投入计划':'数字人力计划投入',
    
    '数字创新论文发表数量':'数字创新论文',
    '数字发明专利授权量': '数字发明专利',
    '数字创新资质认定数量': '数字创新资质',
    '数字化国家级奖项数量':'数字国家级奖项',
    "客户集中度": "客户集中度(%)", "供应商集中度": "供应商集中度(%)",
    '固定资产数字化': "硬件投入占固定资产比重(%)",
    '无形资产数字化': "软件投入占无形资产比重(%)",
    "供应链合作伙伴数量":"供应链合作伙伴"
}
result = dataset.rename(columns=rename_map).copy(deep=True)
indicators = [
    '股票代码', '股票简称', '截止日期', '行业代码', '所属省份', '股权性质',
    "wufei", "zhaochenyv", "yuanchun", "lishouxi", "csmar",
    '净资产收益率(%)', '资产报酬率(%)', 'EBITDA率(%)','营业收入增长率(%)','成本费用利润率(%)',
    '总资产周转率(%)', '应收账款周转率(%)', '存货周转率(%)', '营业周期', '现金资产比率(%)',
    '技术人员占比(%)', '研发营收比(%)', '发明专利申请数', '数字化软硬件投入比(%)',
    '管理层数字职务设立', '数字化战略导向前瞻性', '数字化战略导向持续性', '数字化战略导向广度', '数字化战略导向强度', '数字发明专利','数字国家级奖项', '数字创新论文', '数字创新标准', '数字创新资质',
    '两权分离率(%)', '数字人力计划投入', '数字资本计划投入', '科技创新基地建设',
    '员工人均营收比(%)', '提供岗位增长率(%)', '员工收入增长率(%)',  '社会责任报告质量',
    '供应链合作伙伴', '数字化供应链覆盖度', '客户集中度(%)', '供应商集中度(%)',
]
# '投入资本回报率(%)',  '总资产增长率(%)', '资本积累率(%)', '营业利润增长率(%)'
#    '股权集中度(%)', '独董比例(%)','现金流动负债比率(%)',  '董事会规模', '资产负债率(%)', '权益乘数(%)', '速动比率(%)', '流动资产周转率(%)','长期资本负债率(%)'
result = result[indicators].dropna(subset=indicators)
result = result[(result['截止日期'] > 2017)&(result['截止日期']<2024)]
stocks = list(set(result['股票代码'].tolist()))
print(f"获取到2017年以后共计{len(stocks)}家企业。")

units = []
for stock in stocks:
    cdata = result[result['股票代码'] == stock]
    if len(cdata) >5:
        units.append(stock)
result = result[result['股票代码'].isin(units)].sort_values(['股票代码', '截止日期'])
# result.shape[0]

print(f'共获得2017年以后持续5年{len(set(result["股票代码"]))}家企业无缺失值样本数据{result.shape[0]}个')
# "股票简称", '行业代码', '所属省份','股权性质',
result.drop(["wufei", "zhaochenyv", "yuanchun", "lishouxi", "csmar"], axis=1, inplace=True)
result = result.reset_index(drop=True).sort_values(["股票代码", "截止日期"])
result.head(10)
result.to_csv('./data/制造企业绩效评价数据总表.csv', index=False)


获取到2017年以后共计2455家企业。
共获得2017年以后持续5年1436家企业无缺失值样本数据8616个
